In [19]:
%load_ext autoreload

%autoreload 2

%load_ext snakeviz
import yaml
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy.stats import chi2
import multiprocessing as mp

from ruspy.data.data_reading import data_reading
from ruspy.data.data_processing import data_processing

from ruspy.estimation.estimation import estimate
from ruspy.estimation.estimation_cost_parameters import calc_fixp
from ruspy.estimation.estimation_cost_parameters import lin_cost
from ruspy.estimation.estimation_cost_parameters import cost_func
from ruspy.estimation.estimation_cost_parameters import create_transition_matrix

from ruspy.simulation.simulation import simulate
from ruspy.simulation.robust_sim import get_worst_trans

from ruspy.plotting.value_zero import discount_utility
from ruspy.plotting.value_zero import calc_ev_0
from ruspy.simulation.robust_sim import draw_trans_probs_mulitvar, calc_cov_multinomial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [2]:
# Prepare the raw data.
data_reading()

In [22]:
# Beta is set almost to one, as the agents objective is to maximize average cost.
beta = 0.9999
# 200 buses should be enough to gurantee convergence.
num_buses = 200
# Set the number of simulated periods to 80000. The first plot shows the convergence at this point.
num_periods = 80000
seed = 123
# Paramters for plotting
gridsize = 1000
num_points = int(num_periods/gridsize)

#Initilize a dictionary with values needed on the way.
init_dict = {
    'estimation':{
        'groups': 'group_4',
        'beta': beta,
        'maint_func': 'linear',
        'binsize': 5000,
        'states': 90,
    },
    'simulation':{
        'beta': beta,
        'periods': num_periods,
        'seed': seed,
        'maint_func': 'linear',
        'buses': num_buses
    }
    
}
np.random.seed(init_dict['simulation']['seed'])

In [6]:
repl_data = data_processing(init_dict['estimation'])

In [7]:
trans_results, cost_results = estimate(init_dict['estimation'], repl_data)
print(trans_results, cost_results)

{'state_count': array([[68, 94,  9, ...,  0,  0,  0],
       [ 0, 44, 59, ...,  0,  0,  0],
       [ 0,  0, 47, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  1,  0],
       [ 0,  0,  0, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  0,  0,  1]]), 'x': array([0.39189189, 0.59529357, 0.01281454]), 'fun': 3140.5705570938244, 'trans_count': [1682, 2555, 55]}       fun: 163.585839925182
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-9.94759830e-05,  3.41060513e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 12
   status: 0
  success: True
        x: array([10.07770591,  2.29413601])


In [28]:
p_ml = trans_results['x']
init_dict['simulation']['known trans'] = p_ml
init_dict['simulation']['params'] = cost_results['x']

In [29]:
num_states = 200
costs = cost_func(num_states, lin_cost, init_dict['simulation']['params'])

In [14]:
roh = chi2.ppf(0.95, len(p_ml) - 1) / (2 * 4292)
roh

0.0006979804924403517

In [16]:
worst_trans = get_worst_trans(init_dict['simulation'], p_ml, roh, num_states, choice=True, max_it=10000)

In [17]:
worst_trans

array([0.37416087, 0.61169369, 0.01414543])

In [25]:
trans_mat_worst = create_transition_matrix(num_states, worst_trans)
ev_worst = calc_fixp(num_states, trans_mat_worst, costs, beta)

In [26]:
trans_mat_ml = create_transition_matrix(num_states, p_ml)
ev_ml = calc_fixp(num_states, trans_mat_ml, costs, beta)

In [24]:
n = 4292
probs = draw_trans_probs_mulitvar(n, p_ml, size=100, seed=seed)

In [43]:
def evaluate_strategies(p):
    print(p)
    init_dict_real = copy.deepcopy(init_dict['simulation'])
    init_dict_real['real trans'] = p
    init_dict_real['ev known'] = ev_ml
    df, unobs, utilities, num_states = simulate(init_dict_real)
    v_ml = discount_utility([0., 0.], num_buses, num_periods, num_periods, utilities, beta)[-1]
    init_dict_real['ev known'] = ev_worst
    df, unobs, utilities, num_states = simulate(init_dict_real)
    v_worst = discount_utility([0., 0.], num_buses, num_periods, num_periods, utilities, beta)[-1]
    return v_worst - v_ml

In [44]:
pool = mp.Pool(4)
performance = pool.map(evaluate_strategies, probs)

[0.40048095 0.58421551 0.01530354]
[0.39428832 0.59189567 0.01381601]
[0.39906348 0.58636399 0.01457253]
[0.38166888 0.60718626 0.01114486]
[0.39175696 0.59583049 0.01241255]
[0.38990879 0.59317021 0.016921  ]
[0.40354087 0.58457344 0.0118857 ]
[0.40172393 0.58650265 0.01177342]
[0.41022313 0.57759504 0.01218183]
[0.38271103 0.60067487 0.0166141 ]
[0.39000975 0.59638024 0.01361001]
[0.40258688 0.58477809 0.01263503]
[0.39807079 0.5860777  0.0158515 ]
[0.38424001 0.60487809 0.0108819 ]
[0.39889429 0.58941939 0.01168632]
[0.39620601 0.59577171 0.00802228]
[0.39074668 0.5946048  0.01464852]
[0.38454593 0.60337613 0.01207794]
[0.38140701 0.60693854 0.01165445]
[0.39626898 0.58929487 0.01443615]
[0.37760935 0.60708397 0.01530669]
[0.39948041 0.59042563 0.01009396]
[0.39319726 0.59018371 0.01661904]
[0.39127515 0.59472925 0.0139956 ]
[0.39692278 0.58886555 0.01421167]
[0.39555265 0.5887528  0.01569455]
[0.38412657 0.60243888 0.01343455]
[0.39048982 0.59808996 0.01142022]
[0.39516132 0.593874

In [46]:
np.savetxt("performance.txt", performance)

In [47]:
performance

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]